Automation of finding the lines of best fit based on Mean Average Error (MAE)

## 1. Import Libraries

In [1]:
from ipywidgets import interact
from fastai.basics import *
import pandas as pd
from functools import partial

## 2. Upload Data

In [99]:
df = pd.read_csv("upload_dataset.csv")
x_trch = torch.tensor(df.x)
y_trch = torch.tensor(df.y)

## 3. Create Customisable Quadratic functions and Interactively Plot with MAE

In [101]:
def gen_quad_fn(a,b,c,x): return a*x**2 + b*x + c
def custom_quad_fn(a,b,c): return partial(gen_quad_fn,a,b,c)
# def mae(prediction, actual): return np.mean(abs(prediction-actual))
# def torch_mae(prediction, actual): return np.mean(torch.abs(prediction-actual))
def torch_mae(prediction, actual): return (torch.abs(prediction-actual).mean())
def mae(prediction, actual): return (torch.abs(prediction-actual).mean())
def mae2(prediction, actual): return abs(prediction-actual).mean()
# def mae_jh(prediction, actual): return (abs(prediction-actual)).mean()
# def mse_jh(prediction, actual): return ((prediction-actual)**2).mean()
# def mae(preds, acts): return (torch.abs(preds-acts)).mean()

In [69]:
    # >>> torch.abs(torch.tensor([-1, -2, 3]))


Docstring:
abs(input, *, out=None) -> Tensor

Computes the absolute value of each element in :attr:`input`.

.. math::
    \text{out}_{i} = |\text{input}_{i}|

Args:
    input (Tensor): the input tensor.

Keyword args:
    out (Tensor, optional): the output tensor.

Example::

    >>> torch.abs(torch.tensor([-1, -2, 3]))
    tensor([ 1,  2,  3])
Type:      builtin_function_or_method


In [102]:
plt.rc('figure', dpi=90)

@interact(a=(0,5,0.1),b=(0,5,0.1),c=(0,5,0.1))
def interactive_plot2(a,b,c):
# 1.    plot scatter
    plt.scatter(df.x, df.y)
# x = torch.linspace(-2, 2, steps=20)[:,None]
# 2     create custom_quad_interactive_fn
# 2.1   create xs_interact    
    # xs_interact = torch.linspace(-2.1,2.1,100)
    xs_interact = df.x

# 3.    create ys_interact
    plt.ylim(-1,15)
    ys_interact = custom_quad_fn(a,b,c)(xs_interact)

# 4.    calc mae
    y_actual     = torch.tensor(df.y)
    y_predicted  = torch.tensor(custom_quad_fn(a,b,c)(df.x))
    interact_mae = torch_mae(y_predicted,y_actual)
    # interact_mae = mae_jh(y_predicted,y_actual)

# 5. plot   
    plt.plot(xs_interact, ys_interact)
    plt.title(f"MAE: {interact_mae:.2f}")


interactive(children=(FloatSlider(value=2.0, description='a', max=5.0), FloatSlider(value=2.0, description='b'…

## 4. Determining the effect of the parameters ($a$, $b$, $c$) in:  $ax + bx^2 + c$

The key thing to understand if whether the loss function gets better or worse when you increase the parameters a little.

There are two ways we can try:
1. **Manually** adjust the parameter: Move each parameter each way and observe the impact to MAE.  
2. Calculate the **Derivative** of the parameter: A Derivative iS a function that tells you if you increase the input the: 
    - **direction** in which output changes (increases or decreases) and the;  
    - **magnitude** of the change to the output

### 4.1 Create Mean-Absolute-Error (mae) Quadratic Function
This function will take in the parameters or coefficients of a quadratic function and output the MSE.
- *Input*: coeffiicents of quadratic
- *Output*: MAE (between the prediction of the quadratic with the coffecients of the quadratic and the actual predictsions)

In [103]:
def mae_quad_fn(params):
    quad_fn = custom_quad_fn(*params)
    y_predicted_trch = torch.tensor(quad_fn(df.x))
    y_actual_trch    = torch.tensor(df.y)
    # so quad_params(2,3,4) ->  creates a custom quad fn -> 2x^2 + 3x + 4
    return torch_mae(y_predicted_trch,y_actual_trch)


The chart shows MAE(2,2,2) = 1.45 loss

In [104]:
mae_quad_fn([2,2,2])

tensor(1.4501, dtype=torch.float64)

A **tensor** is a pytorch type that works with: 
- lists (1D tensors)
- tables (2D tensors)
- layers of tables of numbers (3D tensors) and etc

### 4.2 Telling PyTorch to calculate gradients

tensor([2., 2., 2.], requires_grad=True)

#### 4.2.1 Testing it out

In [106]:
def mae_quad_fn2(x_trch, y_trch, params_trch):
    quad_fn_trch = custom_quad_fn(*params_trch)
    y_predicted_trch = quad_fn_trch(x_trch)
    y_actual_trch    = y_trch
    # so quad_params(2,3,4) ->  creates a custom quad fn -> 2x^2 + 3x + 4
    return torch_mae(y_predicted_trch,y_actual_trch)

# rank 1 tensor
abc = torch.tensor([2.0,2.0,2.0])
abc.requires_grad_()



tensor([2., 2., 2.], requires_grad=True)

In [107]:
x_trch = torch.tensor(df.x)
y_trch = torch.tensor(df.y)

loss = mae_quad_fn2(x_trch, y_trch, abc)
loss

tensor(1.4501, dtype=torch.float64, grad_fn=<MeanBackward0>)

![](int_plot_mae.jpg)

## To be Continued...  

The next section go through a more automated method to find the smallest MAE.

[[TBA] Part 1]()   
[[TBA] Part 2]()